In [1]:
import os
import pandas as pd
import httpx
import asyncio
import time
from itertools import batched

In [2]:
indexes = (
    ("WORLD/STANDARD/BLEND", "WORLD", "990100"),
    ("WORLD/STANDARD/VALUE", "WORLD VALUE", "105868"),
    ("WORLD/STANDARD/GROWTH", "WORLD GROWTH", "105867"),
    ("WORLD/SMALL/BLEND", "WORLD SMALL CAP", "106230"),
    ("WORLD/SMALL/VALUE", "WORLD SMALL VALUE", "655131"),
    ("WORLD/SMALL/GROWTH", "WORLD SMALL GROWTH", "655236"),
    ("WORLD/SMID/BLEND", "WORLD SMID CAP", "661668"),
    ("WORLD/SMID/VALUE", "WORLD SMID VALUE", "661773"),
    ("WORLD/SMID/GROWTH", "WORLD SMID GROWTH", "661878"),
    ("WORLD/MID/BLEND", "WORLD MID CAP", "652555"),
    ("WORLD/MID/VALUE", "WORLD MID VALUE", "652660"),
    ("WORLD/MID/GROWTH", "WORLD MID GROWTH", "652758"),
    ("WORLD/LARGE/BLEND", "WORLD LARGE CAP", "650038"),
    ("WORLD/LARGE/VALUE", "WORLD LARGE VALUE", "650143"),
    ("WORLD/LARGE/GROWTH", "WORLD LARGE GROWTH", "650248"),
    ("WORLD/IMI/BLEND", "WORLD IMI", "664185"),
    ("WORLD/IMI/VALUE", "WORLD IMI VALUE", "664283"),
    ("WORLD/IMI/GROWTH", "WORLD IMI GROWTH", "664395"),
    ("ACWI/STANDARD/BLEND", "ACWI", "892400"),
    ("ACWI/STANDARD/VALUE", "ACWI VALUE", "106039"),
    ("ACWI/STANDARD/GROWTH", "ACWI GROWTH", "106038"),
    ("ACWI/SMALL/BLEND", "ACWI SMALL CAP", "655045"),
    ("ACWI/SMALL/VALUE", "ACWI SMALL VALUE", "655150"),
    ("ACWI/SMALL/GROWTH", "ACWI SMALL GROWTH", "655255"),
    ("ACWI/SMID/BLEND", "ACWI SMID CAP", "661687"),
    ("ACWI/SMID/VALUE", "ACWI SMID VALUE", "661792"),
    ("ACWI/SMID/GROWTH", "ACWI SMID GROWTH", "661897"),
    ("ACWI/MID/BLEND", "ACWI MID CAP", "652574"),
    ("ACWI/MID/VALUE", "ACWI MID VALUE", "652679"),
    ("ACWI/MID/GROWTH", "ACWI MID GROWTH", "652784"),
    ("ACWI/LARGE/BLEND", "ACWI LARGE CAP", "650057"),
    ("ACWI/LARGE/VALUE", "ACWI LARGE VALUE", "650162"),
    ("ACWI/LARGE/GROWTH", "ACWI LARGE GROWTH", "650267"),
    ("ACWI/IMI/BLEND", "ACWI IMI", "664204"),
    ("ACWI/IMI/VALUE", "ACWI IMI VALUE", "664309"),
    ("ACWI/IMI/GROWTH", "ACWI IMI GROWTH", "664414"),
    ("EM (EMERGING MARKETS)/STANDARD/BLEND", "EM (EMERGING MARKETS)", "891800"),
    ("EM (EMERGING MARKETS)/STANDARD/VALUE", "EM (EMERGING MARKETS) VALUE", "106063"),
    ("EM (EMERGING MARKETS)/STANDARD/GROWTH", "EM (EMERGING MARKETS) GROWTH", "106062"),
    ("EM (EMERGING MARKETS)/SMALL/BLEND", "EM (EMERGING MARKETS) SMALL CAP", "655061"),
    (
        "EM (EMERGING MARKETS)/SMALL/VALUE",
        "EM (EMERGING MARKETS) SMALL VALUE",
        "655166",
    ),
    (
        "EM (EMERGING MARKETS)/SMALL/GROWTH",
        "EM (EMERGING MARKETS) SMALL GROWTH",
        "655271",
    ),
    ("EM (EMERGING MARKETS)/SMID/BLEND", "EM (EMERGING MARKETS) SMID CAP", "661703"),
    ("EM (EMERGING MARKETS)/SMID/VALUE", "EM (EMERGING MARKETS) SMID VALUE", "661808"),
    (
        "EM (EMERGING MARKETS)/SMID/GROWTH",
        "EM (EMERGING MARKETS) SMID GROWTH",
        "661913",
    ),
    ("EM (EMERGING MARKETS)/MID/BLEND", "EM (EMERGING MARKETS) MID CAP", "652590"),
    ("EM (EMERGING MARKETS)/MID/VALUE", "EM (EMERGING MARKETS) MID VALUE", "652695"),
    ("EM (EMERGING MARKETS)/MID/GROWTH", "EM (EMERGING MARKETS) MID GROWTH", "652800"),
    ("EM (EMERGING MARKETS)/LARGE/BLEND", "EM (EMERGING MARKETS) LARGE CAP", "650073"),
    (
        "EM (EMERGING MARKETS)/LARGE/VALUE",
        "EM (EMERGING MARKETS) LARGE VALUE",
        "650178",
    ),
    (
        "EM (EMERGING MARKETS)/LARGE/GROWTH",
        "EM (EMERGING MARKETS) LARGE GROWTH",
        "650283",
    ),
    ("EM (EMERGING MARKETS)/IMI/BLEND", "EM (EMERGING MARKETS) IMI", "664220"),
    ("EM (EMERGING MARKETS)/IMI/VALUE", "EM (EMERGING MARKETS) IMI VALUE", "664325"),
    ("EM (EMERGING MARKETS)/IMI/GROWTH", "EM (EMERGING MARKETS) IMI GROWTH", "664430"),
    ("SINGAPORE/STANDARD/BLEND", "SINGAPORE", "998100"),
    ("SINGAPORE/STANDARD/VALUE", "SINGAPORE VALUE", "105812"),
    ("SINGAPORE/STANDARD/GROWTH", "SINGAPORE GROWTH", "105811"),
    ("SINGAPORE/SMALL/BLEND", "SINGAPORE SMALL CAP", "106224"),
    ("SINGAPORE/SMALL/VALUE", "SINGAPORE SMALL VALUE", "655127"),
    ("SINGAPORE/SMALL/GROWTH", "SINGAPORE SMALL GROWTH", "655232"),
    ("SINGAPORE/SMID/BLEND", "SINGAPORE SMID CAP", "661664"),
    ("SINGAPORE/SMID/VALUE", "SINGAPORE SMID VALUE", "661769"),
    ("SINGAPORE/SMID/GROWTH", "SINGAPORE SMID GROWTH", "661874"),
    ("SINGAPORE/MID/BLEND", "SINGAPORE MID CAP", "652551"),
    ("SINGAPORE/MID/VALUE", "SINGAPORE MID VALUE", "652656"),
    ("SINGAPORE/MID/GROWTH", "SINGAPORE MID GROWTH", "652761"),
    ("SINGAPORE/LARGE/BLEND", "SINGAPORE LARGE CAP", "650034"),
    ("SINGAPORE/LARGE/VALUE", "SINGAPORE LARGE VALUE", "650139"),
    ("SINGAPORE/LARGE/GROWTH", "SINGAPORE LARGE GROWTH", "650244"),
    ("SINGAPORE/IMI/BLEND", "SINGAPORE IMI", "664181"),
    ("SINGAPORE/IMI/VALUE", "SINGAPORE IMI VALUE", "664286"),
    ("SINGAPORE/IMI/GROWTH", "SINGAPORE IMI GROWTH", "664391"),
    ("USA/STANDARD/BLEND", "USA", "984000"),
    ("USA/STANDARD/VALUE", "USA VALUE", "105826"),
    ("USA/STANDARD/GROWTH", "USA GROWTH", "105825"),
    ("USA/SMALL/BLEND", "USA SMALL CAP", "106229"),
    ("USA/SMALL/VALUE", "USA SMALL VALUE", "655133"),
    ("USA/SMALL/GROWTH", "USA SMALL GROWTH", "655238"),
    ("USA/SMID/BLEND", "USA SMID CAP", "661670"),
    ("USA/SMID/VALUE", "USA SMID VALUE", "661775"),
    ("USA/SMID/GROWTH", "USA SMID GROWTH", "661880"),
    ("USA/MID/BLEND", "USA MID CAP", "652557"),
    ("USA/MID/VALUE", "USA MID VALUE", "652662"),
    ("USA/MID/GROWTH", "USA MID GROWTH", "652767"),
    ("USA/LARGE/BLEND", "USA LARGE CAP", "650040"),
    ("USA/LARGE/VALUE", "USA LARGE VALUE", "650145"),
    ("USA/LARGE/GROWTH", "USA LARGE GROWTH", "650250"),
    ("USA/IMI/BLEND", "USA IMI", "664187"),
    ("USA/IMI/VALUE", "USA IMI VALUE", "664292"),
    ("USA/IMI/GROWTH", "USA IMI GROWTH", "664397"),
    (
        "KOKUSAI INDEX (WORLD ex JP)/STANDARD/BLEND",
        "KOKUSAI INDEX (WORLD ex JP)",
        "991200",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/STANDARD/VALUE",
        "KOKUSAI INDEX (WORLD ex JP) VALUE",
        "105850",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/STANDARD/GROWTH",
        "KOKUSAI INDEX (WORLD ex JP) GROWTH",
        "105849",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/SMALL/BLEND",
        "KOKUSAI INDEX (WORLD ex JP) SMALL CAP",
        "106239",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/SMALL/VALUE",
        "KOKUSAI INDEX (WORLD ex JP) SMALL VALUE",
        "655118",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/SMALL/GROWTH",
        "KOKUSAI INDEX (WORLD ex JP) SMALL GROWTH",
        "655223",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/SMID/BLEND",
        "KOKUSAI INDEX (WORLD ex JP) SMID CAP",
        "661655",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/SMID/VALUE",
        "KOKUSAI INDEX (WORLD ex JP) SMID VALUE",
        "661760",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/SMID/GROWTH",
        "KOKUSAI INDEX (WORLD ex JP) SMID GROWTH",
        "661865",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/MID/BLEND",
        "KOKUSAI INDEX (WORLD ex JP) MID CAP",
        "652542",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/MID/VALUE",
        "KOKUSAI INDEX (WORLD ex JP) MID VALUE",
        "652647",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/MID/GROWTH",
        "KOKUSAI INDEX (WORLD ex JP) MID GROWTH",
        "652752",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/LARGE/BLEND",
        "KOKUSAI INDEX (WORLD ex JP) LARGE CAP",
        "650025",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/LARGE/VALUE",
        "KOKUSAI INDEX (WORLD ex JP) LARGE VALUE",
        "650130",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/LARGE/GROWTH",
        "KOKUSAI INDEX (WORLD ex JP) LARGE GROWTH",
        "650235",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/IMI/BLEND",
        "KOKUSAI INDEX (WORLD ex JP) IMI",
        "664172",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/IMI/VALUE",
        "KOKUSAI INDEX (WORLD ex JP) IMI VALUE",
        "664277",
    ),
    (
        "KOKUSAI INDEX (WORLD ex JP)/IMI/GROWTH",
        "KOKUSAI INDEX (WORLD ex JP) IMI GROWTH",
        "664382",
    ),
    ("JAPAN/STANDARD/BLEND", "JAPAN", "939200"),
    ("JAPAN/STANDARD/VALUE", "JAPAN VALUE", "105796"),
    ("JAPAN/STANDARD/GROWTH", "JAPAN GROWTH", "105795"),
    ("JAPAN/SMALL/BLEND", "JAPAN SMALL CAP", "106218"),
    ("JAPAN/SMALL/VALUE", "JAPAN SMALL VALUE", "655117"),
    ("JAPAN/SMALL/GROWTH", "JAPAN SMALL GROWTH", "655222"),
    ("JAPAN/SMID/BLEND", "JAPAN SMID CAP", "661654"),
    ("JAPAN/SMID/VALUE", "JAPAN SMID VALUE", "661759"),
    ("JAPAN/SMID/GROWTH", "JAPAN SMID GROWTH", "661864"),
    ("JAPAN/MID/BLEND", "JAPAN MID CAP", "652541"),
    ("JAPAN/MID/VALUE", "JAPAN MID VALUE", "652646"),
    ("JAPAN/MID/GROWTH", "JAPAN MID GROWTH", "652751"),
    ("JAPAN/LARGE/BLEND", "JAPAN LARGE CAP", "650024"),
    ("JAPAN/LARGE/VALUE", "JAPAN LARGE VALUE", "650129"),
    ("JAPAN/LARGE/GROWTH", "JAPAN LARGE GROWTH", "650234"),
    ("JAPAN/IMI/BLEND", "JAPAN IMI", "664171"),
    ("JAPAN/IMI/VALUE", "JAPAN IMI VALUE", "664276"),
    ("JAPAN/IMI/GROWTH", "JAPAN IMI GROWTH", "664381"),
    ("WORLD ex USA/STANDARD/BLEND", "WORLD ex USA", "991000"),
    ("WORLD ex USA/STANDARD/VALUE", "WORLD ex USA VALUE", "105874"),
    ("WORLD ex USA/STANDARD/GROWTH", "WORLD ex USA GROWTH", "105873"),
    ("WORLD ex USA/SMALL/BLEND", "WORLD ex USA SMALL CAP", "106238"),
    ("WORLD ex USA/SMALL/VALUE", "WORLD ex USA SMALL VALUE", "655138"),
    ("WORLD ex USA/SMALL/GROWTH", "WORLD ex USA SMALL GROWTH", "655243"),
    ("WORLD ex USA/SMID/BLEND", "WORLD ex USA SMID CAP", "661675"),
    ("WORLD ex USA/SMID/VALUE", "WORLD ex USA SMID VALUE", "661780"),
    ("WORLD ex USA/SMID/GROWTH", "WORLD ex USA SMID GROWTH", "661885"),
    ("WORLD ex USA/MID/BLEND", "WORLD ex USA MID CAP", "652562"),
    ("WORLD ex USA/MID/VALUE", "WORLD ex USA MID VALUE", "652667"),
    ("WORLD ex USA/MID/GROWTH", "WORLD ex USA MID GROWTH", "652772"),
    ("WORLD ex USA/LARGE/BLEND", "WORLD ex USA LARGE CAP", "650045"),
    ("WORLD ex USA/LARGE/VALUE", "WORLD ex USA LARGE VALUE", "650150"),
    ("WORLD ex USA/LARGE/GROWTH", "WORLD ex USA LARGE GROWTH", "650255"),
    ("WORLD ex USA/IMI/BLEND", "WORLD ex USA IMI", "664192"),
    ("WORLD ex USA/IMI/VALUE", "WORLD ex USA IMI VALUE", "664297"),
    ("WORLD ex USA/IMI/GROWTH", "WORLD ex USA IMI GROWTH", "664402"),
)

In [3]:
index_variants = {"GRTR": "Gross", "NETR": "Net"}

In [4]:
requests = [
    (*index, *index_variant, *date_range)
    for index in indexes
    for index_variant in index_variants.items()
    for date_range in (
        ("19691231", "20001130"),
        (
            "20001229",
            (pd.Timestamp.now() + pd.tseries.offsets.BDay(-1, True)).strftime("%Y%m%d"),
        ),
    )
]
async with httpx.AsyncClient() as client:
    tasks = (
        client.get(
            "https://app2-nv.msci.com/products/service/index/indexmaster/downloadLevelData",
            params={
                "output": "INDEX_LEVELS",
                "currency_symbol": "USD",
                "index_variant": index_variant,
                "start_date": start_date,
                "end_date": end_date,
                "data_frequency": "END_OF_MONTH",
                "baseValue": "false",
                "index_codes": code,
            },
            timeout=20,
        )
        for path, index_name, code, index_variant, tax_treatment, start_date, end_date in requests
    )
    for task_batch, request_batch in zip(batched(tasks, 8), batched(requests, 8)):
        response_batch = await asyncio.gather(*task_batch)
        for response, (
            path,
            index_name,
            code,
            index_variant,
            tax_treatment,
            start_date,
            end_date,
        ) in zip(response_batch, request_batch):
            if not response.is_success:
                print(
                    f"Error downloading {index_name} {tax_treatment} {start_date}-{end_date} data"
                )
                continue
            os.makedirs(
                os.path.join(os.getcwd(), "data", "MSCI", *path.split("/")),
                exist_ok=True,
            )
            with open(
                os.path.join(
                    os.getcwd(),
                    "data",
                    "MSCI",
                    *path.split("/"),
                    f"MSCI {index_name} USD {tax_treatment} Monthly {start_date}-{end_date}.xls",
                ),
                "wb",
            ) as f:
                f.write(response.content)
        time.sleep(1)

In [6]:
requests = [
    (*index, *index_variant, *date_range)
    for index in indexes
    for index_variant in index_variants.items()
    for date_range in zip(
        pd.date_range("2001-01-01", "today", freq="5BYS-JAN"),
        pd.date_range("2005-01-01", "today", freq="5BYE-DEC").union(
            [pd.to_datetime("today") + pd.tseries.offsets.BMonthEnd(-1, True)]
        ),
    )
]
async with httpx.AsyncClient() as client:
    tasks = (
        client.get(
            "https://app2-nv.msci.com/products/service/index/indexmaster/downloadLevelData",
            params={
                "output": "INDEX_LEVELS",
                "currency_symbol": "USD",
                "index_variant": index_variant,
                "start_date": start_date.strftime("%Y%m%d"),
                "end_date": end_date.strftime("%Y%m%d"),
                "data_frequency": "DAILY",
                "baseValue": "false",
                "index_codes": code,
            },
            timeout=20,
        )
        for path, index_name, code, index_variant, tax_treatment, start_date, end_date in requests
    )
    for task_batch, request_batch in zip(batched(tasks, 8), batched(requests, 8)):
        response_batch = await asyncio.gather(*task_batch)
        for response, (
            path,
            index_name,
            code,
            index_variant,
            tax_treatment,
            start_date,
            end_date,
        ) in zip(response_batch, request_batch):
            if not response.is_success:
                print(
                    f"Error downloading {index_name} {tax_treatment} {start_date}-{end_date} data"
                )
                continue
            os.makedirs(
                os.path.join(os.getcwd(), "data", "MSCI", *path.split("/")),
                exist_ok=True,
            )
            with open(
                os.path.join(
                    os.getcwd(),
                    "data",
                    "MSCI",
                    *path.split("/"),
                    f"MSCI {index_name} USD {tax_treatment} Daily {start_date.strftime("%Y%m%d")}-{end_date.strftime("%Y%m%d")}.xls",
                ),
                "wb",
            ) as f:
                f.write(response.content)
        time.sleep(1)